In [ ]:
from shapely.geometry import Point, Polygon
from matplotlib import pyplot as plt
from math import sin, cos, sqrt, atan2, radians

import shapely
import pickle
import random
import numpy as np
import gmaps, os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from ipywidgets.embed import embed_minimal_html
import webbrowser
# The functions of Geoguessr class are loaded 
# from the geoLSTM.py file
import neural_network

# directory containing data
DATADIR = "/data"
DATACOMBINED = DATADIR + "/ready_data/"
POLYDIR = "/data" 
# directory to store model in
MODELDIR = "data/models"

In [ ]:
# load training image file names
TF = np.load(DATADIR + '/ready_data/training.npy')
# load testing image file names
TESF = np.load(DATADIR +'/ready_data/testing.npy')
print("Train files: {}, Test files: {}".format(len(TF), len(TESF)))

In [ ]:
# load split grid file
usaPolyGrid = pickle.load(open(POLYDIR + "/usaPolyGrid.pkl",'rb'))
for i in usaPolyGrid.values():
    plt.plot(i[:,1],i[:,0], color='black')
print(len(usaPolyGrid))

In [ ]:
geoModel = geoLSTM.Geoguessr(useRestnet=False)

In [ ]:
geoModel.fit(TF, DATACOMBINED, 
      saveFolder=MODELDIR, 
      batchSize=4, 
      epochs=2,  
      plot=True)

In [ ]:
# modelName = '/restnet_5.738_19.h5'
modelName = '/nonrestNet_5.744_12.h5'
geoModel = geoLSTM.Geoguessr.load(MODELDIR + modelName)

In [ ]:
geoModel.predictSingle(TESF[0], DATACOMBINED, ployGrid=usaPolyGrid)

In [ ]:
geoModel.evaluate(TESF, DATACOMBINED, usaPolyGrid)

In [ ]:
models = []
for i in os.listdir(MODELDIR):
    models.append(geoLSTM.Geoguessr.load(MODELDIR + '/' +i))

In [ ]:
trainableParams = []
avgDistanceError = []
for gm in models:
    trainableParams.append(sum([tf.keras.backend.count_params(i) for i in gm.model.trainable_weights]))
    avgDistanceError.append(gm.evaluate(TESF, DATACOMBINED, usaPolyGrid))

In [ ]:
plt.bar(np.arange(len(trainableParams)), height=avgDistanceError)
plt.xticks(np.arange(len(trainableParams)), trainableParams)
plt.xlabel("Trainabel paramers per model")
plt.ylabel("Average miles")
plt.title("Average miles per model")
plt.show()